In [1]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.5/905.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
from elasticsearch import Elasticsearch

In [3]:
import getpass
es_cloud_id = getpass.getpass("Enter Elastic Cloud ID: ")
api_id = getpass.getpass("Enter Elastic API ID: ")
api_key = getpass.getpass("Enter Elastic API KEY: ")

# Create the client instance
client = Elasticsearch(cloud_id=es_cloud_id, api_key=(api_id, api_key))
client.info()

Enter Elastic Cloud ID: ··········
Enter Elastic API ID: ··········
Enter Elastic API KEY: ··········


ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '5089524370ab4e1396e3270ca581c541', 'cluster_uuid': 'LFC-6cmGSsqshnnawgItBg', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
es_model_id = 'intfloat__multilingual-e5-base'

In [48]:
def simple_search(query):

    response = client.search(
        index="site",
        knn={
            "field": "information_embedding.predicted_value",
            "query_vector_builder": {
                "text_embedding": {
                    "model_id": es_model_id,
                    "model_text": f"query: {query}"
                }
            },
            "k": 5,
            "num_candidates": 20,
        }
    )

    formatted_results = []
    for hit in response["hits"]["hits"]:
        result = {
            "score": hit["_score"],
            "관광지명": hit["_source"]["name"],
            "주소": hit["_source"]["address"],
            "관광지설명": hit["_source"]["information"]
        }
        formatted_results.append(result)

    return formatted_results

In [54]:
simple_search("충청도 문화 체험")

[{'score': 0.9373121,
  '관광지명': '증평민속체험박물관',
  '주소': '충청북도 증평군 증평읍 남하리 91',
  '관광지설명': '충청북도 증평군 : 증평민속체험박물관 : 증평의 역사와 전통문화를 만날 수 있는 문화공간으로 증평의 문화재, 옛 사진, 출토유물, 고서, 민속품 등 다양한 문화유산을 관람할 수 있으며, 농경문화체험과 전시연계교육 등 종합적인 교육체험을 할 수 있음'},
 {'score': 0.93590117,
  '관광지명': '충주호체험',
  '주소': '충청북도 충주시 중앙탑면 탑평리 545-1',
  '관광지설명': '충청북도 충주시 : 충주호체험 : 역사 문화와 수변공간이 어우러지는 체험관광지 조성 / 수변의 위락시설과 생태환경의 조화로 체험가능한 수변 휴식 학습공간 조성 /수상레저활성화를 위한 국제수변레포츠단지 조성'},
 {'score': 0.935225,
  '관광지명': '공주문화관광지',
  '주소': '충청남도 공주시 웅진동 431',
  '관광지설명': '충청남도 공주시 : 공주문화관광지 : 백제역사 유적관광지'},
 {'score': 0.9338298,
  '관광지명': '서동요역사관광지',
  '주소': '충청남도 부여군 충화면 가화리 313',
  '관광지설명': '충청남도 부여군 : 서동요역사관광지 : 역사문화환경을 구축하여 부여 대표 관광명소'},
 {'score': 0.9322176,
  '관광지명': '서산버드랜드',
  '주소': '',
  '관광지설명': '충청남도 서산시 : 서산버드랜드 : 철새도래시기에는 천수만에 도래하는 철새들을 탐조투어를 통해 볼 수 있고 특별프로그램 기간에는 각종 체험행사 등을 경험할 수 있으며 4D 영상 등의 볼거리가 있음'}]

In [57]:
simple_search("전북 자연 관광")

[{'score': 0.9348736,
  '관광지명': '완주전통문화체험장',
  '주소': '',
  '관광지설명': '전북특별자치도 완주군 : 완주전통문화체험장 : 고즈넉한 한옥의 숨결을 느끼며 뛰어난 자연경관 속에 아늑한 휴식을 즐기기에 안성맞춤인 공간'},
 {'score': 0.934268,
  '관광지명': '금강호관광지',
  '주소': '전북특별자치도 군산시 성산면 성덕리 411-1',
  '관광지설명': '전북특별자치도 군산시 : 금강호관광지 : 국내 최대 철새도래지로서 자연과 문화가 어우러지는 사계절 관광지로 거듭나고 있음'},
 {'score': 0.93109846,
  '관광지명': '고산자연휴양림',
  '주소': '',
  '관광지설명': '전북특별자치도 완주군 : 고산자연휴양림 : 숲에서 산림욕을 즐길 수 있는 사계절 가족휴양지'},
 {'score': 0.93071413,
  '관광지명': '변산해수욕장',
  '주소': '전북특별자치도 부안군 변산면 대항리 523-117',
  '관광지설명': '전북특별자치도 부안군 : 변산해수욕장 : 국립공원 변산반도에 위치한 노을이 아름다운 해수욕장'},
 {'score': 0.92856264,
  '관광지명': '웅포관광지',
  '주소': '전북특별자치도 익산시 웅포면 웅포리 727',
  '관광지설명': '전북특별자치도 익산시 : 웅포관광지 : 웅포 곰개나루 및 캠핑장'}]

In [58]:
simple_search("부산 박물관 관람")

[{'score': 0.9300904,
  '관광지명': '부산영화체험박물관',
  '주소': '부산광역시 중구 동광동3가 41-4',
  '관광지설명': '부산광역시 중구 : 부산영화체험박물관 : 부산영화체험박물관은 대한민국을 대표하는 영화체험 복합문화공간입니다. 전국 최초의 영화관련 전문전시체험시설로 지상지상 4층, 지하 3층으로 구성된 본 체험박물관은 영화의 원리를 이해하고 제작의 즐거움을 체험할 수 있는 다양한 전시시설 및 영화의 과거와 미래를 연결하는 하이테크 시네마를 갖추고 있습니다.'},
 {'score': 0.9264631,
  '관광지명': '복천박물관',
  '주소': '부산광역시 동래구 복천동 16-5',
  '관광지설명': '부산광역시 동래구 : 복천박물관 : 삼한시대부터 삼국시대까지 부산의 역사를 보여주는 고고전문박물관으로서, 현재 7차에 걸친 발굴조사로 밝혀진 169기의 유구와 각종 토기 및 철기류 등의 유물을 종합 전시하고있다'},
 {'score': 0.92395544,
  '관광지명': '해양자연사박물관',
  '주소': '부산광역시 동래구 온천동 산 13-1',
  '관광지설명': '부산광역시 동래구 : 해양자연사박물관 : 1994년 6월 10일 동래구 온천동 금강공원 내에 개관한 부산해양자연사박물관은 전국 최초, 최대규모의 해양자연사분야의 전문박물관으로'},
 {'score': 0.92253256,
  '관광지명': '부평역사박물관',
  '주소': '인천광역시 부평구 삼산동 451-1',
  '관광지설명': '인천광역시 부평구 : 부평역사박물관 : 부평 역사관련 전시물 관람'},
 {'score': 0.9217539,
  '관광지명': '영도관광안내센터',
  '주소': '부산광역시 영도구 봉래동1가 1',
  '관광지설명': '부산광역시 영도구 : 영도관광안내센터 : 부산의 새로운 관광명소로 각광받고있는 영도대교의 도개모습을 감상하고 영도관광에 필요한 종합서비스를 제공받을 수 있다.'}]